# Hierarchical clustering program using different distance measures

Use pm4py for importing different event logs found in logs folder

In [ ]:
import pandas as pd
import pm4py
import networkx as nx
import numpy as np
from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.spatial.distance import squareform
import pprint

def get_log(path):
    '''
    Reads in an event log from the logs folder using pm4py 
    '''

    return pm4py.read_xes(path)

def get_filtered_log(path, num_top_k):
    filtered_log = get_log(path)
    filtered_log = pm4py.filter_variants_top_k(filtered_log, num_top_k)

    return filtered_log

# fetch the information out of an event log using pm4py
full_log = get_log("../logs/sepsis_event_log.xes")
filtered_log = get_filtered_log("../logs/sepsis_event_log.xes", num_top_k=62)
highly_filtered_log = get_filtered_log("../logs/sepsis_event_log.xes", num_top_k=11)

Different helper methods

In [ ]:
def print_log_info(log, verbose=False):
    '''
    Prints out useful information of the given event log

    Prints out all start activities, end activities, all event attributes, and all trace attributes.
    If verbose is True, then all values of all event and trace attributes are printed out plus all variants of the event log
    '''

    start_activities = pm4py.stats.get_start_activities(log)
    end_activities = pm4py.stats.get_end_activities(log)
    event_attributes = pm4py.stats.get_event_attributes(log)
    trace_attributes = pm4py.stats.get_trace_attributes(log)
    
    info = f"Event log information:\n"
    info += f"Start activities: {start_activities}\n"
    info += f"End activities: {end_activities}\n"
    info += f"Event attributes: {event_attributes}\n"
    info += f"Trace attributes: {trace_attributes}\n"

    # add additional info if wished for
    if verbose:
        event_attribute_values = [
            {attribute: pm4py.stats.get_event_attribute_values(log, attribute)}
            for attribute in event_attributes
        ]
        trace_attribute_values = [
            {attribute: pm4py.stats.get_trace_attribute_values(log, attribute)}
            for attribute in trace_attributes
        ]
        variants = pm4py.stats.get_variants(log)

        info += f"Event attribute values: {event_attribute_values}\n"
        info += f"Trace attribute values: {trace_attribute_values}\n"
        info += f"Variants: {variants}"

    print(info)

def show_dfg_of_log(log):
    '''
    Displays the directly follows graph of an event log
    '''

    dfg, start_activities, end_activities = pm4py.discovery.discover_dfg(log, 
                                                                         case_id_key= "case:concept:name",
                                                                         activity_key= "concept:name")

    return pm4py.vis.view_dfg(dfg, start_activities, end_activities, format="png")

def get_df_from_dfg(log):
    '''
    Reads in the log and then produces a dfg from it. The function then generates a Pandas DataFrame.

    Every row in the DataFrame has a source node, a target node and the frequency of the vertex
    '''

    dfg, _, _ = pm4py.discover_dfg(log)
    connections_list = [[edges[0], edges[1], weight] for edges, weight in dfg.items()] 
    df = pd.DataFrame(data=connections_list, columns=["From", "To", "Frequency"])

    return df

def get_pivot_df_from_dfg(log):
    '''
    Reads in the log and then produces a dfg from it. The function then generates a Pandas DataFrame.

    The returned DataFrame is a pivot table, rows are source nodes, columns are target nodes and the values is the frequency the vertex has been used 
    '''

    df = get_df_from_dfg(log)

    # find events which are not in ingoing and outgoing events
    from_events = df["From"].unique()
    to_events = df["To"].unique()
    only_from_elements = list(set(from_events) - set(to_events))
    only_to_elements = list(set(to_events) - set(from_events))

    # add them to the dataframe in the column in which they were not in
    # that way all elements in the pivot table are in both column and rows
    for element in only_from_elements:
        df.loc[len(df)] = [from_events[0], element, 0]
    for element in only_to_elements:
        df.loc[len(df)] = [element, to_events[0], 0]

    df = df.pivot(index="From", columns="To", values="Frequency")
    df = df.fillna(0)

    return df

def get_weighted_df(df):
    '''
    Transforms the frequency of the used vertices to weights. The weight is the portion of all outgoing vertices of a node.
    Ex.: Node A has three vertices with frequency 8, 7 and 5. After applying the transformation the weights are 0.4, 0.35 and 0.25

    Returns the normalized df
    '''
    
    normalized_df = df.apply(lambda x: (x / x.sum()), axis="columns")
    normalized_df = normalized_df.fillna(0)

    return normalized_df

def get_jaccard_distance_of_dfg(log):
    '''
    Computes the jaccard measure of two nodes using the DFG

    NetworkX provides an implementation of the jaccard measure in directly-follows-graphs.
    First the DFG has to be generated using pm4py. The output gets formatted for a NetworkX graph.
    Output is an iterator of the pairwise distances of the nodes in the graph
    '''

    # convert event log to an undirected NetworkX graph
    dfg, start, end = pm4py.discover_dfg(log)
    edge_list = [(edge[0], edge[1], weight) for edge, weight in dfg.items()]
    dg = nx.DiGraph()
    dg.add_weighted_edges_from(edge_list)

    # compute the jaccard measures of the nodes
    distances = nx.jaccard_coefficient(dg.to_undirected())

    # print distances between the nodes
    #   for distance in distances:
    #        print(distance)

    return distances

def calculate_weighted_jaccard_similarity(G, node1, node2):
    '''
    Calculates the weighted jaccard distance between two nodes of a Graph
    Formula is the sum of the minimum weights of the union of vertices
    divited by the sum of the maximum weights of the union of vertices

    Source: https://stackoverflow.com/a/69218150
    '''
    
    # skip calculating the similarity if the nodes to compare are the same
    if node1 == node2:
        return 1

    neighbors1 = set(G.neighbors(node1))
    neighbors2 = set(G.neighbors(node2))
    minimums = 0
    maximums = 0

    for x in neighbors1.union(neighbors2):
        node1_weight = 0
        node2_weight = 0

        if x in G[node1]:
            node1_weight = G[node1][x]['weight']
        if x in G[node2]:
            node2_weight = G[node2][x]['weight']
        
        minimums += min(node1_weight, node2_weight)
        maximums += max(node1_weight, node2_weight)
    # prevent division by 0 error
    if maximums == 0:
        return 0
    return minimums / maximums

def get_weighted_jaccard_distance_matrix(connections_df):
    '''
    Computes the weighted jaccard distance for every node pair

    Returns a pivoted DataFrame where the row is the source node,
    the column the target node and the values are the distances between the nodes 
    '''
    
    G = nx.from_pandas_adjacency(connections_df, nx.DiGraph)
    jaccard_table = {} # the distances between the nodes

    for node1 in G.nodes:
        node1_distances = {}
        for node2 in G.nodes:
            similarity = calculate_weighted_jaccard_similarity(G, node1, node2)
            node1_distances[node2] = round(1 - similarity, 6)
        jaccard_table[node1] = node1_distances

    jaccard_table = pd.DataFrame(jaccard_table)
    return jaccard_table

def get_simrank_similarity_of_dfg(log):
    '''
    Computes the simrank similarity of two nodes using the DFG

    Output is a dictionary of dictionaries with the first key being the source node, the key
    in the second dictionary being the target node, and the value being the simrank similarity.
    The distance relation between source and target are symmetrical. 
    '''

    # convert event log to a NetworkX graph
    dfg, start, end = pm4py.discover_dfg(log)
    edge_list = [(edge[0], edge[1], weight) for edge, weight in dfg.items()]
    dg = nx.DiGraph()
    dg.add_weighted_edges_from(edge_list)

    # compute the simrank similarity of the nodes
    distances = nx.simrank_similarity(dg)

    # print distances between the nodes
    #   pprint.pprint(distances)

    return distances

def convert_jaccard_distances_to_distance_matrix(log, pairwise_distances):
    '''
    Converts the pairwise distances iterator from NetworkX to a distance matrix

    First finds the attributes and generates an empty distance matrix.
    The cells will be filled with 1 - the values of the corresponding jaccard measure.
    Remaining cells have the distance 1
    Returns the list of activities and the distance matrix
    '''
    
    # retrieve all attribute values
    activities = list(pm4py.get_event_attribute_values(log, "concept:name").keys())
    activities.sort()

    # create empty distance matrix using the activity names    
    distance_matrix = pd.DataFrame(columns=activities, index=activities)

    # fill matrix up
    for u, v, distance in pairwise_distances:
        distance_matrix[u][v] = 1 - round(distance, 3)
        distance_matrix[v][u] = 1 - round(distance, 3)

    distance_matrix = distance_matrix.fillna(1) # no similarity -> distance = 1
    distance_matrix = distance_matrix.to_numpy()
    np.fill_diagonal(distance_matrix, 0) # similarity to itself is always 0
    
    # return only the values of the matrix, not the names of the activities
    return activities, distance_matrix

def convert_simrank_distances_to_distance_matrix(log, pairwise_distances):
    '''
    Converts the pairwise distances dictionary of dictionaries to a distance matrix
    '''

    # create distance matrix using the pairwise distances dictionary and sort them
    distance_matrix = pd.DataFrame(pairwise_distances)
    distance_matrix = distance_matrix.reindex(sorted(distance_matrix.columns), axis="index")
    distance_matrix = distance_matrix.reindex(sorted(distance_matrix.columns), axis="columns")
    activities = distance_matrix.columns

    # transform values and convert to numpy matrix
    distance_matrix = distance_matrix.transform(lambda x: round(1 - x, 3))
    print(distance_matrix)
    distance_matrix = distance_matrix.to_numpy()
    
    # return only the values of the matrix, not the names of the activities
    return activities, distance_matrix

def create_linkage(distances):
    '''
    Performs hierarchical clustering using average linkage with scipy
    Returns the linkage matrix
    '''

    # input for the method has to be a condensed distance matrix
    condensed_matrix = squareform(distances)

    # perform hierarchical clustering
    return linkage(condensed_matrix, 'average')

def create_dendrogram(activities, linkage_matrix):
    '''
    Creates the dendrogram of the hierarchical clustering operation using scipy
    Displays the dendrogram to the terminal
    '''

    # create dendrogram out of data
    dn = dendrogram(linkage_matrix,
                    orientation="right",
                    labels=activities)

    return dn

def create_clusterings_for_every_level(activities, distances, linkage_matrix):
    '''
    Creates a dictionary for every clustering. Each entry includes a list of clusterings 
    for every level of the dendrogram.
    Returns the clustering dictionary
    '''

    # output dictionary and the number of done merges 
    clusterings = {}
    num_merges = 0

    # add a singleton cluster for every activity at first entry in clusterings dictionary
    dct = dict([(i, {activities[i]}) for i in range(distances.shape[0])])
    clusterings[0] = list(dct.values())

    # adapted from: https://stackoverflow.com/a/65060545
    for i, row in enumerate(linkage_matrix, distances.shape[0]):
        # add for every merge a union of the merged clusters and delete the old ones
        dct[i] = dct[row[0]].union(dct[row[1]])
        del dct[row[0]]
        del dct[row[1]]
        num_merges += 1

        # save the clustering in the output dictionary
        clusterings[num_merges] = list(dct.values())
    
    return clusterings

def create_hierarchy_for_activities(activities, clusterings):
    '''
    Returns every cluster each activity was ever in. This in return generates the hierarchy needed
    for abstraction of event logs
    Hint: Currect runtime is O(|activities|^2), there maybe is a faster method, though this is fine for now
    '''

    # create empty hierarchy lists for each activity
    hierarchies = {activity: [] for activity in activities}

    # for each cluster in each level add the new cluster to the corresponding activities
    for level in clusterings.values():
        for cluster in level:
            for activity in cluster:
                act_hierarchy = hierarchies[activity]
                # skip cluster if it is already in its hierarchy
                if cluster in act_hierarchy:
                    continue
                act_hierarchy.append(cluster)

    return hierarchies


In [ ]:
def weighted_simrank(G, node1, node2, C=0.8, max_iterations=100, tolerance=1e-6):
    if node1 == node2:
        return 1.0

    prev_sim = 0
    sim = 1
    iterations = 0

    while abs(sim - prev_sim) > tolerance and iterations < max_iterations:
        prev_sim = sim
        neighbors1 = set(G.predecessors(node1))
        neighbors2 = set(G.predecessors(node2))
        sim = C / (len(neighbors1) * len(neighbors2)) * sum(G[node1][x]['weight'] * G[node2][y]['weight'] * weighted_simrank(G, x, y, C, max_iterations, tolerance) for x in neighbors1 for y in neighbors2)
        iterations += 1

    return sim

def full_weighted_simrank(G):
    similarities = {}
    for node1 in G.nodes():
        node_similarity = {}
        for node2 in G.nodes():
            node_similarity[node2] = weighted_simrank(G, node1, node2)
        similarities[node1] = {}


In [ ]:
def perform_clustering_using_weighted_jaccard(log):
    connections_df = get_pivot_df_from_dfg(log)
    connections_df = get_weighted_df(connections_df)

    jaccard_distances = get_weighted_jaccard_distance_matrix(connections_df)
    jaccard_activities = jaccard_distances.columns
    display(jaccard_distances)
    
    jaccard_linkage = create_linkage(jaccard_distances.to_numpy())
    result_dendrogram = create_dendrogram(jaccard_activities, jaccard_linkage)
    
    clusterings = create_clusterings_for_every_level(jaccard_activities, jaccard_distances, jaccard_linkage)
    hierarchies = create_hierarchy_for_activities(jaccard_distances, clusterings)


In [ ]:
coselog = get_log("../logs/coselog.xes")
perform_clustering_using_weighted_jaccard(coselog)

In [ ]:
bpic = get_log("../logs/BPI_Challenge_2013_incidents.xes")
perform_clustering_using_weighted_jaccard(bpic)

In [ ]:
road_traffic = get_log("../logs/Road_Traffic_Fine_Management_Process.xes")
perform_clustering_using_weighted_jaccard(road_traffic)

In [ ]:
perform_clustering_using_weighted_jaccard(full_log)

In [ ]:
# TESTING
import SimRank # imported from https://github.com/ysong1231/SimRank/tree/main

connections_df_full = get_df_from_dfg(full_log)
connections_df_filtered = get_df_from_dfg(filtered_log)
#connections_df_full["Frequency"] = connections_df_full.groupby("From")["Frequency"].transform(lambda x: (x / x.sum()))

def perform_weighted_simrank(connections):
    sr = SimRank.SimRank()
    sr_distances = sr.fit(data=connections,
                        verbose=False,
                        weighted = True,
                        from_node_column="From",
                        to_node_column="To",
                        weight_column="Frequency")
    sr_distances = sr_distances.transform(lambda x: 1 - x)
    display(sr_distances)

    sr_activities = sr_distances.columns
    sr_distances = sr_distances.to_numpy()


    # Apply min-max normalization to the entire array
    np.fill_diagonal(sr_distances, np.nan)
    min_val = np.nanmin(sr_distances)
    max_val = np.nanmax(sr_distances)
    normalized_data = np.vectorize(lambda x: (x - min_val) / (max_val - min_val))(sr_distances)
    np.fill_diagonal(normalized_data, 0)
    sr_distances = normalized_data

    sr_linkage = create_linkage(sr_distances)
    result_dendrogram = create_dendrogram(sr_activities, sr_linkage)

#display(connections_df_full.pivot(index="From", columns="To", values="Frequency").fillna(0))
perform_weighted_simrank(connections_df_full)


Main program for executing the generalisation hierarchies

In [ ]:
def show_all_information_from_log(log):
    # Compute the pairwise distances of the created DFG from the event log
    # pairwise_distances = get_jaccard_distance_of_dfg(log)
    pairwise_distances = get_simrank_similarity_of_dfg(log)

    # displays the directly follows graph of the log
    print("The generated directly-follows-graph of the log:")
    dfg = show_dfg_of_log(log)
    print("------------------------------------------------------")

    # Generate the distance matrix used for performing hierarchical clustering
    # also prints the distance matrix while executing
    print("Distances between the activities:")
    # activities, distances = convert_jaccard_distances_to_distance_matrix(log, pairwise_distances)
    activities, distances = convert_simrank_distances_to_distance_matrix(log, pairwise_distances)
    print("------------------------------------------------------")

    # Perform hierarchical clustering using scipy
    linkage_matrix = create_linkage(distances)

    # create a dictionary for every clustering performed at each step
    clusterings = create_clusterings_for_every_level(activities, distances, linkage_matrix)

    # create the hierarchy needed for event log abstraction
    print("The clusters each activity was in:")
    hierarchies = create_hierarchy_for_activities(activities, clusterings)
    for activity, clusters in hierarchies.items():
        print(f"{activity}: ")
        for cluster in clusters:
            print(f"\t{cluster}")
    print("------------------------------------------------------")
    
    # create a dendrogram for illustrative purposes
    print("The resulting dendrogram of the activities:")
    result_dendrogram = create_dendrogram(activities, linkage_matrix)
    print("------------------------------------------------------")



The information of the full unfiltered log:

In [ ]:
show_all_information_from_log(full_log)

The information from the filtered log (only variants which occur twice will be considered)

In [ ]:
show_all_information_from_log(filtered_log)

The information from the highly filtered log (only variants which occur five times will be considered)

In [ ]:
show_all_information_from_log(highly_filtered_log)